In [7]:
#第六章：数据加载、存储与文件格式
#读取文本文件和其他更高效的磁盘存储格式，加载数据库中的数据，利用Web API操作网络资源
#读写文本格式的数据
type examples/ex1.csv

SyntaxError: invalid syntax (<ipython-input-7-89bd296b22b4>, line 4)

In [10]:
#使用read_csv将其读入一个DataFrame：
import pandas as pd
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
#使用read_table,并指定分隔符：
pd.read_table('examples/ex1.csv',sep= ',')

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
pd.read_csv('examples/ex2.csv',header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [14]:
#自己定义行列名
pd.read_csv('examples/ex2.csv',names = ['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
#将列message指定为索引：通过index_col参数指定“message”：
names = ['a','b','c','d','message']
pd.read_csv('examples/ex2.csv',names = names, index_col ='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [21]:
#将多个列作为一个层次化索引
parsed = pd.read_csv('examples/ex3.csv',index_col = ['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

AttributeError: 'list' object has no attribute 'to_csv'

In [27]:
list(open('examples/ex4.txt'))

["' A B C',\n",
 "'aaa -0.264438 -1.026059 -0.619500',\n",
 "'bbb 0.927272 0.302904 -0.032399',\n",
 "'ccc -0.264273 -0.386314 -0.217601',\n",
 "'ddd -0.871858 -0.348382 1.100491'"]

In [28]:
result = pd.read_table('examples/ex4.txt', sep='\s+')
result

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,',A,B,"C',"
0,'aaa,-0.264438,-1.026059,"-0.619500',"
1,'bbb,0.927272,0.302904,"-0.032399',"
2,'ccc,-0.264273,-0.386314,"-0.217601',"
3,'ddd,-0.871858,-0.348382,1.100491'


In [33]:
pd.read_csv('examples/ex6.csv',skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [35]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [37]:
result = pd.read_csv('examples/ex5.csv',na_values = ['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [40]:
sentinels= {'message':['foo','NA'],'something':['two']}
result = pd.read_csv('examples/ex5.csv',na_values =sentinels)
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [41]:
#逐块读取文本文件
pd.options.display.max_rows = 10
result = pd.read_csv('examples/ex6.csv')
result

,one two three four key
0,0 0.467976 -0.038649 -0.295344 -1.824726 L
1,1 -0.358893 1.404453 0.704965 -0.200638 B
2,2 -0.501840 0.659254 -0.421691 -0.057688 G
3,3 0.204886 1.074134 1.388361 -0.982404 R
4,4 0.354628 -0.133116 0.283763 -0.837063 Q
...,...
6,9995 2.311896 -0.417070 -1.409599 -0.515821 L
7,9996 -0.479893 -0.650419 0.745152 -0.646038 E
8,9997 0.523331 0.787112 0.486066 1.093156 K
9,9998 -0.362559 0.598894 -1.843201 0.887292 G


In [42]:
#指定行：
pd.read_csv('examples/ex6.csv',nrows = 5)

,one two three four key
0,0 0.467976 -0.038649 -0.295344 -1.824726 L
1,1 -0.358893 1.404453 0.704965 -0.200638 B
2,2 -0.501840 0.659254 -0.421691 -0.057688 G
3,3 0.204886 1.074134 1.388361 -0.982404 R
4,4 0.354628 -0.133116 0.283763 -0.837063 Q


In [43]:
#逐块读取文件，可以指定chunksize（行数）：
chunker = pd.read_csv('examples/ex6.csv',chunksize = 1000)
chunker

In [49]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value = 0)
tot = tot.sort_values(ascending = False)
tot[:10]

Series([], dtype: float64)

In [48]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]
#TextParser还有一个get_chunk方法，它使你可以读取任意大小的块

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [50]:
#将数据写出到文本格式
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [51]:
#利用DataFrame的to_csv方法
data.to_csv('examples/out.csv')

In [55]:
!type examples\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [ ]:
#在linux中访问文件使用：!cat ch06/ex1.csv  #使用'/'
#在windows命令行中使用：!type ch06/ex1.csv #使用‘\’；加上绝对路径时加引号

In [56]:
import sys
#直接写出到sys.stdout，所以仅仅是打印出文本结果
data.to_csv(sys.stdout,sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [57]:
#缺失值转换为其他标签
data.to_csv(sys.stdout,na_rep = 'NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [58]:
#禁用标签：
data.to_csv(sys.stdout,index= False,header = False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [59]:
#并以你指定的顺序排序：
data.to_csv(sys.stdout,index =False,columns = ['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [63]:
#Series也有一个to_csv方法
dates = pd.date_range('1/1/2000',periods = 7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [4]:
import numpy as np
import pandas as pd
dates = pd.date_range('1/1/2000', periods=7)
#ts = pd.Series(np.arange(7),index = dates)
#ts
ts = pd.Series(np.arange(7), index=dates)

In [5]:
ts.to_csv('examples/tseries.csv')
#ts.to_csv('examples/tseries.csv')

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [8]:
#处理分隔符格式
!type examples\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [9]:
#对于任何单字符分隔符文件，可使用csv模块
import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)
for line in reader:
    print(line)
#移除所有的引号，产生元组

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [10]:
#读取文件到一个多行的列表中
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
#将行分为标题行和数据行
header,values = lines[0],lines[1:]
#使用字典构造式和zip(*values),后者将行转置为列，创建数据列的字典
data_dict = {h: v for h, v in zip(header,zip(*values))

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [14]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [ ]:
#class my_dialect(csv.Dialect):
#    delimiter = ';'
#    quotechar = '"'
#    quoting = csv.QUOTE_MINIMAL
#reader = csv.reader(f, dialect=my_dialect)
#reader = csv.reader(f,delimiter = '|')
#reader = csv.reader(f, delimiter='|')

In [ ]:
#对于那些使用复杂分隔符或多字符分隔符的文件，csv模块就无能为力了。这种情
#况下，你就只能使用字符串的split方法或正则表达式方法re.split进行行拆分和其他整理工
#作了

In [15]:
#手工输出分隔符文件，可使用csv.writer
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))